In [2]:
import nibabel as nib
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from einops.layers.torch import Rearrange
import torch
import torchvision
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import copy


def normalize_frame(frame):
    """Normalize the frame data to 0-255."""
    frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)
    return frame.astype(np.uint8)

# Load the image
path = r"C:\Users\razva\Master1\Thesis\Data\ADNI_preprocessed\train\I273707_ASL_PERFUSION_MoCo_20110321082031_7.nii.gz"
img = nib.load(path)
data = img.get_fdata()
# data = np.zeros(shape=[64, 64, 48, 105])
print(data.shape)

if len(data.shape) < 4:
    data = np.expand_dims(data, axis=-1)

print(data.shape)

TARGET_SPACE_DIM = [64, 64, 48]
zoom_factor = (TARGET_SPACE_DIM[0] / data.shape[0], TARGET_SPACE_DIM[1] / data.shape[1], TARGET_SPACE_DIM[2] / data.shape[2], 1)

TARGET_TIME_DIM = 140

print(data.shape)

crt_space_dim = list(data.shape[:3])
crt_time_dim = data.shape[-1]

# if crt_time_dim >= TARGET_TIME_DIM:
#     data = data[:, :, :, :TARGET_TIME_DIM]
# else:
#     padded_data = np.zeros(shape=(crt_space_dim + [TARGET_TIME_DIM - crt_time_dim]))
#     print(padded_data.shape)
#     data = np.concatenate([data, padded_data], axis=3)

print(data.shape)
    
# data = scipy.ndimage.zoom(data, zoom_factor)

print(data.shape)

# Extract shape dynamically
x_dim, y_dim, z_dim, time_stamps = data.shape

# Ensure frame size is even for OpenCV compatibility
frame_size = 256
frame_size = frame_size if frame_size % 2 == 0 else frame_size - 1  

num_cols = int(np.ceil(np.sqrt(time_stamps)))
num_rows = int(np.ceil(time_stamps / num_cols))

dest_folder = "movies"
os.makedirs(dest_folder, exist_ok=True)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

for slice_index in range(z_dim):
    video_path = os.path.join(dest_folder, f"slice_{slice_index}.mp4")
    out = cv2.VideoWriter(video_path, fourcc, 10, (frame_size, frame_size), isColor=True)

    for temp_index in range(time_stamps):
        frame = data[:, :, slice_index, temp_index]
        frame = normalize_frame(frame)

        rot_frame = np.rot90(frame)
        resized_frame = cv2.resize(rot_frame, (frame_size, frame_size))
        
        # Convert grayscale to BGR explicitly
        color_frame = cv2.cvtColor(resized_frame, cv2.COLOR_GRAY2BGR)
        
        out.write(color_frame)  # Ensure frames are written

    out.release()

print("All videos saved successfully.")

(32, 32, 24, 70)
(32, 32, 24, 70)
(32, 32, 24, 70)
(32, 32, 24, 70)
(32, 32, 24, 70)
All videos saved successfully.


In [17]:
path = r"C:\Users\razva\Master1\Thesis\train_data.csv"
csv = pd.read_csv(path)
csv["File Path"].to_list()

['.\\Data\\ADNI\\128_S_4653\\MoCoSeries\\2012-04-18_15_42_04.0\\I298612_nii\\I298612_ASL_PERFUSION_MoCo_20120418154205_7.nii.gz',
 '.\\Data\\ADNI\\128_S_2045\\MoCoSeries\\2011-03-21_08_20_31.0\\I273707_nii\\I273707_ASL_PERFUSION_MoCo_20110321082031_7.nii.gz',
 '.\\Data\\ADNI\\002_S_4473\\Axial_rsfMRI__Eyes_Open_\\2017-03-07_14_08_04.0\\I827363_nii\\I827363_Axial_rsfMRI_(Eyes_Open)_20170307140804_15.nii.gz',
 '.\\Data\\ADNI\\032_S_6055\\Axial_rsfMRI__Eyes_Open_\\2017-07-24_14_19_53.0\\I879572_nii\\I879572_Axial_rsfMRI_(Eyes_Open)_20170724141954_14.nii.gz',
 '.\\Data\\ADNI\\073_S_2264\\relCBF\\2012-02-28_12_23_02.0\\I286952_nii\\I286952_ASL_PERFUSION_MoCo_20120228122302_10.nii.gz',
 '.\\Data\\ADNI\\035_S_4414\\Axial_rsfMRI__Eyes_Open_\\2017-10-04_10_37_01.0\\I915043_nii\\I915043_Axial_rsfMRI_(Eyes_Open)_20171004103702_12.nii.gz',
 '.\\Data\\ADNI\\019_S_4680\\Extended_Resting_State_fMRI\\2014-04-16_13_24_55.0\\I420457_nii\\I420457_Extended_Resting_State_fMRI_20140416132455_701.nii.gz',
 '